# Compare Methods in Optim.jl

In this notebook we will look into diffterent methods that are currently supported in `Optim.jl` and compare their performances in 5 parts: gradient-free, gradient-required, Hessian-required, linesearch and manifold.

The experiment is based on three problems: Rosenbrock function, Hosaki's and Himmelblau's function. 

In [40]:
# Pkg.add("OptimTestProblems")
using OptimTestProblems
Rosenbrock = MultivariateProblems.UnconstrainedProblems.examples["Rosenbrock"]
Hosaki = MultivariateProblems.UnconstrainedProblems.examples["Hosaki"]
Himmelblau = MultivariateProblems.UnconstrainedProblems.examples["Himmelblau"]

OptimTestProblems.MultivariateProblems.OptimizationProblem{Void,Void,Float64,String,Void}("Himmelblau", OptimTestProblems.MultivariateProblems.UnconstrainedProblems.himmelblau, OptimTestProblems.MultivariateProblems.UnconstrainedProblems.himmelblau_gradient!, nothing, OptimTestProblems.MultivariateProblems.UnconstrainedProblems.himmelblau_hessian!, nothing, [2.0, 2.0], [3.0, 2.0], 0.0, true, true, nothing)

In [4]:
# Pkg.add("Plots")
using Plots; plotlyjs()

Plots.PlotlyJSBackend()

In [23]:
function problem_plot(problem, range)
    x = y = range
    z = Surface((x,y)->problem.f([x,y]), x, y)
    surface(x, y, z, linealpha = 0.3, legend = false)
end

problem_plot (generic function with 1 method)

$$
f({\mathbf  {x}})=f(x_{1},x_{2})= [(1-x_1^2)+100(x_2-x_1^2)]^2
$$

In [24]:
problem_plot(Rosenbrock, -2:0.1:2)

In [31]:
problem_plot(Hosaki, -1:0.1:1)

In [39]:
problem_plot(Himmelblau, -5:0.1:5)

## Gradient-free

In [54]:
# Pkg.add("Optim")
using Optim
result = optimize(Rosenbrock.f, zeros(2), Optim.Options(store_trace = true, extended_trace = true))
trace = Optim.trace(result)
f_trace = Optim.f_trace(result)
hcat(trace...)

1×61 Array{Optim.OptimizationState{Float64,Optim.NelderMead{Optim.AffineSimplexer,Optim.AdaptiveParameters}},2}:
      0     9.506641e-01     4.576214e-02
 * step_type: initial
 * centroid: [0.0125, 0.0]
  …      60     1.876334e-08     9.270314e-09
 * step_type: inside contraction
 * centroid: [1.00006, 1.00012]


In [ ]:
function problem_trace
    

## Gradient-required

## Hessian-required

## Linesearch

## Manifold